In [9]:
import pdb
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from scipy.stats import norm
import statsmodels.api as sm

pd.options.mode.chained_assignment = None

eta = 1
npass = 10
data_path = 'data/pickles/pop_eta%d_first/' % eta
out_path = 'data/new_generated_data/heckman_svm/eta%d/first_run/' % eta
train_file = 'train_clicks_see%d_1.pkl'
test_file = 'test_clicks_see%d_1.pkl'

In [2]:
train = pd.read_pickle(data_path + train_file % 0)
train.head()

,C,S,X1,X10,X100,X101,X102,X103,X104,X105,...,X91,X92,X93,X94,X95,X96,X97,X98,X99,qid
0,1.0,1.0,0.60553,0.93218,0.00000,0.64691,0.98062,0.00000,0.0,0.0,...,0.58327,0.0,0.0,0.00000,0.0,0.00000,0.94384,0.13750,0.94909,1.0
1,0.0,0.0,0.59670,0.95340,0.81380,0.64691,0.98062,0.00000,0.0,0.0,...,0.70022,0.0,0.0,0.82106,0.0,0.00000,0.96996,0.31887,0.94909,1.0
2,0.0,0.0,0.67717,0.86589,0.94901,0.64691,0.98062,0.83723,0.0,0.0,...,0.63190,0.0,0.0,0.86223,0.0,0.74966,0.00000,0.46558,0.94909,1.0
3,0.0,0.0,0.63511,0.89742,0.00000,0.64691,0.98062,0.00000,0.0,0.0,...,0.30710,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.16019,0.94909,1.0
4,0.0,0.0,0.60553,0.97106,0.83358,0.64691,0.95593,0.00000,0.0,0.0,...,0.74024,0.0,0.0,0.78052,0.0,0.00000,0.98023,0.47301,0.94909,1.0


In [3]:
num_files = 1

In [4]:
def probit(Y, X):
# #     lr = LinearRegression()
#     lr = Ridge(alpha=1.0)
#     lr.fit(X, Y)
#     return lr.coef_

    pm = sm.Probit(endog=Y, exog=X, missing='drop')
    pm = pm.fit(method='bfgs', maxiter=50, disp=0)
    return pm.params

def inverse_mills(val):
#     return -norm.pdf(val) / norm.cdf(val)
    return norm.pdf(val) / (1 - norm.cdf(val))

def OLS(Y, X):
    N = X.shape[0]
#     lr = LinearRegression()
    lr = Ridge(alpha=1.0)
    lr.fit(X, Y)
    return lr.coef_

In [7]:
def heckman(train_file):
    train = pd.read_pickle(data_path + train_file)
    train['qid'] = train['qid'].astype(int)
    train['S'] = train['S'].astype(int)
    train['C'] = train['C'].astype(int)

    X = train[train.columns.drop(['qid', 'C', 'S'])]
    Y = train['S']

    gamma = probit(Y, X)
    lambda_ = inverse_mills(-np.matmul(X, gamma))

    XB = X[train['S'] == 1]
    lambda_b = lambda_[train['S'] == 1]
    XBL = np.append(XB, lambda_b.reshape(-1,1), 1)
    Y = train['C'][train['S'] == 1]
    
#     params = OLS(Y, np.append(X, lambda_.reshape(-1,1), 1))[:-1]
    params = probit(Y, XBL)[:-1]
    
    train['cp'] = norm.cdf(np.matmul(X, params))
    
    return train


In [8]:
train = heckman(train_file % 10)
train_see = train[train['S'] == 1]
train_see[['qid', 'C', 'S', 'cp']].head(20)

> <ipython-input-7-c333730e73f1>(11)heckman()
-> gamma = probit(Y, X)


(Pdb)  type(X)


<class 'pandas.core.frame.DataFrame'>


(Pdb)  type(Y)


<class 'pandas.core.series.Series'>


(Pdb)  q


BdbQuit: 

In [142]:
train['cp'].min(), train['cp'].max()

(0.042405978216949997, 0.53819149545433698)

In [136]:
# for i in range(num_files):
#     fname_train = train_file % i
#     heckman(fname_train)